In [3]:
import sys 
sys.path.append('..')

In [4]:
from datetime import datetime as dt

import numpy as np
import pandas as pd

from preprocessing import Preprocessing
from sklearn.model_selection import StratifiedKFold, GridSearchCV


from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [2]:
col_drop = ['CustomerId','Surname']
types = {
    'HasCrCard':bool,
    'IsActiveMember':bool
}
train = pd.read_csv('data/train.csv', index_col='id', dtype=types).drop(columns=col_drop)
train.drop_duplicates(inplace=True)

In [3]:
X = train.drop(columns='Exited')
y = train.Exited

In [4]:
prepro = Preprocessing()
X_scld = prepro.fit_transform(X).astype(float)

In [5]:
skf = StratifiedKFold(n_splits=4)

In [12]:
params = {
    "n_estimators" : [500,800],
    'max_depth':[8,16,32,64],
    'min_samples_split':[32,64,128],
    'min_samples_leaf':[1,2,4,8,16],

}
params = {
    "n_estimators" : [5,10,20]
}
grid = GridSearchCV(
    RandomForestClassifier(),
    params,
    cv=skf,
    scoring='roc_auc',
    n_jobs = -1,
    verbose=10
)

In [13]:
grid.fit(X_scld,y)

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV 2/4; 2/3] START n_estimators=10.............................................
[CV 4/4; 2/3] START n_estimators=10.............................................
[CV 1/4; 3/3] START n_estimators=20.............................................
[CV 3/4; 3/3] START n_estimators=20.............................................
[CV 3/4; 1/3] START n_estimators=5..............................................
[CV 3/4; 2/3] START n_estimators=10.............................................
[CV 2/4; 1/3] START n_estimators=5..............................................
[CV 4/4; 1/3] START n_estimators=5..............................................
[CV 1/4; 1/3] START n_estimators=5..............................................
[CV 1/4; 2/3] START n_estimators=10.............................................
[CV 4/4; 3/3] START n_estimators=20.............................................
[CV 2/4; 3/3] START n_estimators=20..............

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'n_estimators': [5, 10, 20]}, scoring='roc_auc',
             verbose=10)

In [14]:
grid.best_params_

{'n_estimators': 20}

In [15]:
grid.best_score_

0.8600938532634758

In [16]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
2,2.386234,0.087495,0.141316,0.003721,20,{'n_estimators': 20},0.863784,0.861634,0.858840,0.856117,0.860094,0.002889,1
1,1.291760,0.082162,0.087946,0.012074,10,{'n_estimators': 10},0.851111,0.844834,0.843777,0.843359,0.845770,0.003130,2
0,0.731966,0.014766,0.059746,0.007248,5,{'n_estimators': 5},0.820843,0.818043,0.817757,0.818119,0.818690,0.001250,3
